In [973]:
# Docs - https://docs.goong.io/rest/api-key/

In [974]:
# import requests

In [975]:
# from utils.df_handle import *

In [976]:
sql = \
"""
SELECT
BranchID,
SlsperID,
Lat,
Lng,
UpdateTime as checkinupdatetime,
CAST(UpdateTime as date) as checkindate,
CustID,
NumberCICO
from dbo.AR_SalespersonLocationTrace where CAST(UpdateTime as date) >= '2022-04-01' and SlsperID = 'MR0624'
and Type = 'IO'
"""

In [977]:
# df1 = get_ms_df(sql)

In [978]:
# cleancols(df1)

In [979]:
# df1.columns = lower_col(df1)

In [980]:
# df['date']=df.checkinupdatetime.dt.date

In [981]:
# df1['checkindate'] = pd.to_datetime(df1['checkindate'])

In [982]:
df1['checkindate'].unique()

array(['2022-04-01T00:00:00.000000000', '2022-04-03T00:00:00.000000000',
       '2022-04-04T00:00:00.000000000', '2022-04-05T00:00:00.000000000',
       '2022-04-06T00:00:00.000000000', '2022-04-07T00:00:00.000000000',
       '2022-04-08T00:00:00.000000000', '2022-04-11T00:00:00.000000000',
       '2022-04-12T00:00:00.000000000', '2022-04-13T00:00:00.000000000',
       '2022-04-14T00:00:00.000000000', '2022-04-15T00:00:00.000000000',
       '2022-04-16T00:00:00.000000000', '2022-04-18T00:00:00.000000000',
       '2022-04-19T00:00:00.000000000', '2022-04-20T00:00:00.000000000',
       '2022-04-21T00:00:00.000000000', '2022-04-22T00:00:00.000000000',
       '2022-04-23T00:00:00.000000000', '2022-04-24T00:00:00.000000000',
       '2022-04-25T00:00:00.000000000', '2022-04-26T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1034]:
df.head()

,branchid,slsperid,checkinupdatetime,checkindate,custid,numbercico,coord,coord_to,custid_to,numbercico_to


In [1035]:
for i in ['2022-04-25']:
    dk1 = df1['checkindate'] == pd.Timestamp(i)
    df = df1[dk1].copy()
    df.shape
    print("co dong data", df.shape)
    # df.head()
    # df['concat'] = str(df['lat'])+','+str(df['lng'])
    # df['lat'] = df['lat'].astype(str)
    # df['lng'] = df['lng'].astype(str)
    # df.to_clipboard()
    # df['lat'] = df['lat'].astype(str)
    # df['lng'] = df['lng'].astype(str)
    df['coord'] = df['lat'].astype('str') +','+ df['lng'].astype('str')
    # output_str = ''
    df['coord_to'] = df['coord'].shift(-1)
    df['custid_to'] = df['custid'].shift(-1)
    df['numbercico_to'] = df['numbercico'].shift(-1)
    df = df[df.coord_to.notna()]
    drop_cols(df, ['lat', 'lng'])
    # output_str = ''
    origin_str = ''
    for strr in df['coord'].tolist():
        origin_str = origin_str +'|'+strr
    origin_str = origin_str[1:]
    # origin_str
    des_str = ''
    for strr in df['coord_to'].tolist():
        des_str = des_str +'|'+strr
    des_str = des_str[1:]
    # des_str
    # df['lat'].astype('object')+','+df['lng'].astype('object')
    # df.to_clipboard()
    url = \
    f"https://rsapi.goong.io/DistanceMatrix?api_key=leSoeVGgIBHmZ2FZjsmQGIoYa27ZT4ErYvMN8bxF&vehicle=car&origins={origin_str}&destinations={des_str}"
    response = requests.request("GET", url)
    response.json()
    df.head()
    df.shape[0]
    # response.json()['rows'][11]['elements'][11]
    result_lists = []
    for i in range(df.shape[0]):
        rs = response.json()['rows'][i]['elements'][i]
        result_lists.append(rs)
    result_lists
    df_rs = pd.DataFrame(result_lists)
    df_rs.shape
    distance_df = pd.DataFrame(df_rs['distance'].tolist(), index=df_rs.index)
    distance_df.columns = ['distance_text','distance_value']
    duration_df = pd.DataFrame(df_rs['duration'].tolist(), index=df_rs.index)
    duration_df.columns = ['duration_text','duration_value']
    distance_df
    df.reset_index(inplace=True, drop=True)
    df2 = pd.concat([df, distance_df], axis=1)
    # df2.head()
    df3 = pd.concat([df2, duration_df], axis=1)
    # drop_cols(df3, ['lat', 'lng'])
    df3.head()
    bq_values_insert(df3, "f_mds_distance_matrix", 2)

co dong data (13, 8)


1it [00:18, 18.04s/it]


In [ ]:
# Ngay 24, co 1 diem check in thoi, code chua chay duoc